In [38]:
import pandas as pd
import re
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

os.getcwd()
df = pd.read_csv('./dws_dsc_customer_opportunity_monthly_kpi_1.csv')
op = pd.read_csv('dwd_dsc_d365_opportunity_df.csv', sep = '\001')
df.columns = [re.sub('\w+\.', '', i) for i in list(df.columns)]
op.columns = [re.sub('\w+\.', '', i) for i in list(op.columns)]
df = df.fillna(0)
# df.columns
df['if_cooped'] = 0
df['if_cooped'] = df['if_cooped'].where(df['contract_signed_opportunity_num'] < 1, 1) # 今年有没合作。
cus_bf_21 = op[(op['createdon'] < '2021-11-01') & (op['milestone'] == 'Contract Signed')]['account_name_en'].unique()
cus_bf_21 = list(cus_bf_21)
cus_af_21 = list(df['customer_name'].unique())
df['old_cust'] = df['customer_name'].isin(cus_bf_21)
# df.to_csv('./data_down/dws_dsc_customer_opportunity_monthly_kpi_1.csv', index = None, encoding='utf_8_sig')


colors = sns.color_palette('rocket')[0:5]
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# plt1

In [92]:
def plot_pie(plt10, plt11,  bi_title, **kwargs):
    """
    成功, 总和, 
    """
    colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']
    fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
    
    fig.add_trace(
        go.Pie(values = [round(plt10 / plt11, 2), 1- round(plt10 / plt11, 2)], 
        labels = ['succeed', 'fail']),
        1, 1)

    fig.add_trace(
        go.Pie(values = [plt10, plt11 - plt10], 
        labels = ['succeed', 'fail']),
        1, 2)
    fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
    fig.update_layout(title_text=bi_title,
    width=900,
    height=400,
    margin=dict(l=10, r=10, t=50, b=20),
    paper_bgcolor="LightSteelBlue",)

    return fig
 

In [ ]:

# plt10 = sum(df['contract_signed_opportunity_num'])  # win
# plt11 = sum(df['total_opportunity_num']) # 整体的商机成功率.a
# plot_pie(plt10, plt11,  "整体的商机本年成功率\t\n 成功数量")


In [450]:
# plt12 = df.query("total_opportunity_num > 0 & if_cooped == 1")['annual_average_revenue'].sum() # win
# plt13 = df['annual_average_revenue'].sum()
# plot_pie(plt12, plt13,  "整体的商机本年成功金额占比 \t\n 成功金额")

In [90]:
plt20 = df.query("old_cust == 1 & if_cooped == 1 & total_opportunity_num > 0").shape[0] 
plt21 = df.query("old_cust == 1 & total_opportunity_num > 0").shape[0] 
# 0.592375366568915 旧客户留存率。
plot_pie(plt20, plt21, '老客户留存率 \n\t 老客户留存数')

plt30 = df.query("old_cust == 1 & total_opportunity_num > 0 ")['contract_signed_opportunity_num'].sum()
plt31 = df.query("old_cust == 1 & total_opportunity_num > 0 ")['total_opportunity_num'].sum() 
plot_pie(plt30, plt31,  "商机level: 老客本年商机成功率\t\n 成功数量")



plt30 = df.query("old_cust == 1 & total_opportunity_num > 0 & if_cooped == 1")['annual_average_revenue'].sum()
plt31 = df.query("old_cust == 1 & total_opportunity_num > 0 ")['annual_average_revenue'].sum() 
plot_pie(plt30, plt31,  "商机level: 老客本年成功金额占比 \t\n 成功金额")



In [263]:
# df.query("old_cust == 0 & total_opportunity_num > 0 & ")['total_opportunity_num'].sum() 

In [91]:
# [df.query("old_cust == 1")['total_opportunity_num'].sum(), df.query("old_cust == 1")['total_opportunity_num'].sum() / sum(df['total_opportunity_num'])]
plt32 = df.query("old_cust == 0 & total_opportunity_num > 0")['contract_signed_opportunity_num'].sum() 
plt33 = df.query("old_cust == 0 & total_opportunity_num > 0")['total_opportunity_num'].sum()  # xinke zong 
plot_pie(plt32, plt33, '商机level: 新客户本年商机成功率 \n\t 新客户本年商机成功数')

plt36 = df.query("old_cust == 0 & total_opportunity_num > 0 & if_cooped == 1")['annual_average_revenue'].sum()
plt37 = df.query("old_cust == 0 & total_opportunity_num > 0")['annual_average_revenue'].sum()  # xinke zong 
plot_pie(plt36, plt37, '商机level: 新客户本年成功金额占比 \t\n 成功金额')

In [476]:
plt38 = op.query("opportunity_record == 'New Business'")
plt38['annual_average_gross_profit'] = plt38['annual_average_gross_profit'].astype(float)
plt38 = plt38.sort_values('annual_average_gross_profit', ascending=False).head(30)
# px.bar(plt38, x = 'account_name_en', y = 'annual_average_gross_profit')

C:\Users\DSCSHA~1\AppData\Local\Temp/ipykernel_37772/3523455288.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [499]:
# plt39 = op.fillna(0).query("opportunity_record != 0 & opportunity_record != 'BCA Not Required'")
# # plt39 = plt39[plt39['annual_average_gross_profit'].str.contains('[azAZ]').fillna(True)] 
# plt39 = plt39[plt39['annual_average_gross_profit'].str.match('(\d+\.)').fillna(False)]
# plt39['annual_average_gross_profit'] = plt39['annual_average_gross_profit'].astype(float)
# plt39 = plt39.sort_values('annual_average_gross_profit', ascending=False).head(30)
# px.bar(plt39, x = 'account_name_en', y = 'annual_average_gross_profit', color = 'opportunity_record')


---
# 客户

In [28]:
plt42 = len(set(np.intersect1d(cus_bf_21, cus_af_21)))
plt43 = len(cus_af_21)

plot_pie(plt42, plt43, '客户level :  本年有商机的老客占比 \n\t 老客户数')
# """# [341, 717]  今年出现的过往的客户 vs 今年的所有"""

In [30]:
plt40 = df.query("old_cust == 0 & total_opportunity_num > 0 & if_cooped == 1")['customer_name'].nunique()
plt41 = df.query("old_cust == 0 & total_opportunity_num > 0")['customer_name'].nunique()
plot_pie(plt40, plt41, '客户level :  客户转化率 \n\t 客户转化数')


** 从年初到现在  总体 annual_average_revenue**

In [55]:
a0 = df['annual_average_revenue'].sum() ;a0

16164700.52

** 从年初到现在新增的客户为我们带来的 annual_average_revenue 收益 **

In [324]:
a1 = df.query("old_cust == 0 & total_opportunity_num > 0 & if_cooped == 1")['annual_average_revenue'].sum();a1*100

28914133.0

** 从年初到现在老的客户为我们带来的新的商机的 annual_average_revenue 收益 **

In [325]:
b = df.query("old_cust == 1 & total_opportunity_num > 0 & if_cooped == 1")['annual_average_revenue'].sum();b*100

614776910.0

In [56]:
(a1 + b )/ a0 # 百分之四十的金额 被得到.

0.3982078370110132

--- 
## ci

In [41]:
opp = op[['account_name_en','annual_average_gross_profit_base', 'annual_average_gross_profit',
        'annual_average_revenue_base', 'annual_average_revenue',
        'bg', 'contract_term', 'milestone', 'opportunity_record', 'product1', 'sector', 'contract_value',
        'commission', 'fivem_customer']]  
opp = opp.fillna(0)
opp = opp[opp['contract_term'] < 60]
# op['contract_value_log'] = np.log(op['contract_value'])
opp = opp[opp['contract_term'] != 0] 

In [35]:
# opp = op
mst_list = ['Contract Signed', 'Closed-Canceled', 'Closed-Lost']
sctr_list = ['Other', 0]
prodc_list = [0]

# opp = opp[opp['milestone'].isin(mst_list)]
# opp = opp[~opp['sector'].isin(sctr_list)]
# opp = opp[~opp['product1'].isin(prodc_list)]

opp = opp[opp['contract_value'] != 0]
opp['contract_value_log'] = np.log(op['contract_value'])
px.scatter(opp, x = 'contract_term', y = 'contract_value_log',\
     color="sector", trendline = 'ols')


In [67]:
import statsmodels.formula.api as smf
# smf.ols('contract_value ~ contract_term + product1', data = op).fit().summary()


`商机金额大小与转化率的关系。 c 不显著#`

In [78]:
# 商机金额大小与转化率的关系。 c
# df = df[(df['milestone'] != 'Early Lead') & (df['milestone'] != 'Potential Opportunity')]
def load_opp(df):
     df['win_or_loss'] = 0
     df['win_or_loss'] = df['win_or_loss'].where(df['milestone']  != 'Contract Signed', 1)
     df = df.fillna(0)
     # 删除有汉字的。
     df = df[~df['annual_average_gross_profit'].str.contains('[azAZ]').fillna(False)]
     df = df[~df['annual_average_revenue'].str.contains('[azAZ]').fillna(False)]
     return df
opp = load_opp(opp)
smf.logit(formula = "win_or_loss ~ contract_value ", data = opp).fit().summary()

Optimization terminated successfully.
         Current function value: 0.679574
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            win_or_loss   No. Observations:                 5079
Model:                          Logit   Df Residuals:                     5077
Method:                           MLE   Df Model:                            1
Date:                Wed, 08 Dec 2021   Pseudo R-squ.:               0.0001876
Time:                        18:32:38   Log-Likelihood:                -3451.6
converged:                       True   LL-Null:                       -3452.2
Covariance Type:            nonrobust   LLR p-value:                    0.2551
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -0.3250      0.029    -11.256      0.000      -0.382      -0.268
contract_value -8.588e-07   8.37e-07     -1.026      0.305    -2.5e-06    7.81e-07
==================================================================================
"""

In [50]:
opp['milestone'].unique()

array(['Contract Signed', 'Closed-Canceled', 'Early Lead', 'Qualified',
       'Closed-Lost', 'Opportunity Gain', 'Shortlisted',
       'Potential Opportunity', 'DCI Conducted', 'Peer Review'],
      dtype=object)

In [108]:
violin
# px.violin(x = opp['win_or_loss'], y =  np.log(opp['contract_value']), \
#     points="all", color = opp['win_or_loss'], box = True )

### open shangji 
> open 商机可能带来 3565583.7480000006 的收入.

In [87]:
# go.Figure(go.Sunburst(
#     labels=[ "Cain", "Seth", "Enos", "Noam", "Abel", "Awan", "Enoch", "Azura"],
#     parents=[  "Eve", "Eve", "Seth", "Seth", "Eve", "Eve", "Awan", "Eve" ],
#     values=[  14, 12, 10, 2, 6, 6, 4, 4],
# ))

opp[~opp['milestone'].isin(mst_list)]['annual_average_revenue'].astype(float).sum() * .4

open 商机可能带来 3565583.7480000006 的收入.

3565583.7480000006

---
# sap

In [317]:
import pandas as pd
import re
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

os.getcwd()
sap = pd.read_csv('./dws_dsc_customer_operation_monthly_kpi.csv', sep = '\001')
# op = pd.read_csv('dwd_dsc_d365_opportunity_df.csv', sep = '\001')
sap.columns = [re.sub('\w+\.', '', i) for i in list(sap.columns)] 
sap = sap.fillna(0)
sap['ytm_cogs'] = sap['ytm_income_amt'] - sap['ytm_gross_profit_amt']
sap['ytm_gp_margin'] = sap['ytm_gross_profit_amt'] / sap['ytm_income_amt'] 
sap['ytm_gp_margin'].replace([-np.inf, np.inf],  0, inplace = True)
# sap= sap.query('ytm_gp_margin > -3 & ytm_gp_margin < 3 ') 
# df.columns 

In [116]:
# # def plot_pie(plt10, plt11,  bi_title, **kwargs):
# #     """
# #     成功, 总和, 
# #     """
# #     colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']
# #     fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
    
# #     fig.add_trace(
# #         go.Pie(values = [round(plt10 / plt11, 2), 1- round(plt10 / plt11, 2)], 
# #         labels = ['succeed', 'fail']),
# #         1, 1)

# #     fig.add_trace(
# #         go.Pie(values = [plt10, plt11 - plt10], 
# #         labels = ['succeed', 'fail']),
# #         1, 2)
# #     fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
# #                   marker=dict(colors=colors, line=dict(color='#000000', width=2)))
# #     fig.update_layout(title_text=bi_title,
# #     width=900,
# #     height=400,
# #     margin=dict(l=10, r=10, t=50, b=20),
# #     paper_bgcolor="LightSteelBlue",)

# #     return fig
# sap_ly = pd.concat([sap[['customer_name', 'bg_code', 'year', 'month', 'account_name_cn',
#        'geographical_location', 'industry', 'ou_num']], 
#        sap[pd.Series(sap.columns)[pd.Series(sap.columns).str.contains('^ly')]]], axis = 1)
# sap_ly['year'] = 2020
# sap_ty = sap.iloc[:,0:28]
# sap_ly.columns = sap_ty.columns
# sap_long = pd.concat([sap_ty, sap_ly], axis = 0).drop(['month', 'account_name_cn',
#        'geographical_location', 'industry', 'ou_num'], axis = 1)

In [322]:
sap_plt = sap[~sap['customer_name'].str.contains('ISC').fillna(False)]
sap_plt = sap_plt[sap_plt['customer_name'] != 0]
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() 
scaled = scaler.fit_transform(np.array(sap_plt['ytm_income_amt']).reshape(-1,1))
sap_plt['ytm_income_amt_scaled'] = np.log(scaled)
sap_plt['abs_ytm_income_amt'] = np.abs(sap_plt['ytm_income_amt'])
sap_plt= sap_plt.query('ytm_gp_margin > -3 & ytm_gp_margin < 3 ') 
# sap_plt = sap_plt.query("income_amt_scaled > -4")

C:\Users\DSCSHA~1\AppData\Local\Temp/ipykernel_37772/3892181128.py:6: RuntimeWarning:

divide by zero encountered in log



In [398]:
def bubble_plt(plot_data, x, color):
    if x == 'scaled':
        x = 'ytm_income_amt_scaled'
        vline = np.log(scaler.transform(np.array(10000000).reshape(-1,1)))[0][0]
    else:
        x = 'ytm_income_amt'
        vline = 10000000
    # y = y
    fig = px.scatter(
        data_frame= plot_data, x = x, y= 'ytm_gp_margin', hover_data=['customer_name'],
        size = 'abs_ytm_income_amt', color = color, title='gross_profit_margin against income, customer level')  
    
    fig.add_hline(y = .06, line_width=3, line_dash="dash", line_color="black", opacity = .4,\
        annotation_text="2021 <br> GP_margin .06 <br><br />  " , 
        annotation_position="top right", annotation_opacity = .4, col = 1)
    fig.add_vline(x = vline, line_width=3, line_dash="dash", line_color="black", opacity = .4)
    return fig
# bubble_plt(sap_plt, 'x',  'customer_type')

In [330]:
# bubble_plt(sap_plt, 'scaled', 'labor_fee_rate') # 人力成本高中低，对应的客户，其利润率有无显著差异？说明什么? 

In [408]:
# 按人力成本占总成本占比，高中低客户分布如何？
# sap['labor_fee_rate']

smf.ols('labor_fee_rate ~ customer_type', data = sap).fit().summary()


In [405]:
smf.ols('ytm_gp_margin ~ labor_fee_rate', data = sap).fit().summary() #labor_fee

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          ytm_gp_margin   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statistic:                   0.06155
Date:                Wed, 08 Dec 2021   Prob (F-statistic):              0.804
Time:                        21:33:14   Log-Likelihood:                -13778.
No. Observations:                 365   AIC:                         2.756e+04
Df Residuals:                     363   BIC:                         2.757e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3.334e+14   3.24e+14     -1.029      0.304    -9.7e+14    3.04e+14
labor_fee     3.4e+08   1.37e+09      0.248      0.804   -2.36e+09    3.04e+09
==============================================================================
Omnibus:                      862.169   Durbin-Watson:                   2.006
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1991693.422
Skew:                         -19.022   Prob(JB):                         0.00
Kurtosis:                     362.880   Cond. No.                     2.44e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.44e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

6 down


In [409]:
df_plt = df.merge(sap[['labor_fee_rate','labor_fee', 'customer_type', 'customer_name', 'ytm_gp_margin', 'ytm_gross_profit_amt']], on = 'customer_name')

In [411]:
# Q1到Q4不同类别的客户，商机成功率有无显著差异？
# 人力成本不同的客户，商机成功率有无显著差异？


# px.scatter(data_frame= df_plt, x = 'labor_fee', y= 'opportunity_win_rate', hover_data=['customer_name'], 
#         size = 'labor_fee', color = 'customer_type', title='人力费用, 商机胜率, 客户类型')  



# px.violin(x = df_plt['customer_type'], y =  df_plt['labor_fee_rate'], \
#     points="all", color = df_plt['customer_type'], box = True )


    
# px.violin(x = df_plt['customer_type'], y =  df_plt['opportunity_win_rate'], \
#     points="all", color = df_plt['customer_type'], box = True )

In [353]:
# 客户类型 人力成本
# smf.ols('labor_fee_rate ~ customer_type-1', data = df_plt).fit().summary()

In [394]:
# 客户类型, / 人力成本 商机胜率

smf.ols('opportunity_win_rate ~   customer_type - 1 ', data = df_plt).fit().summary()
smf.ols('opportunity_win_rate ~  labor_fee_rate  - 1 ', data = df_plt).fit().summary()

In [413]:
# 人力成本占比越高的业务，越可能产生高的利润? yes
# smf.ols('ytm_gross_profit_amt ~ labor_fee_rate-1 ', data = df_plt).fit().summary() #labor_fee

In [357]:
# p7

In [401]:
# sap_plt['customer_type'].value_counts()

In [370]:
px.bar(data_frame= sap_plt, x = 'customer_type',  hover_data=['customer_name'], 
    color = 'customer_type', title='customer type count')  

# Q4    133
# Q3     80
# 0      47
# Q1     46
# Q2     44

In [417]:
# p9
# bubble_plt(sap_plt, 'scaled', 'is_operation') # has_vas 
# df_plt['has_vas']

In [440]:
# 不同仓储客户的分类
# 有无仓储业务，其利润率高低，有无显著差异？
# 不同仓储业务类型的客户，利润率有无显著差异？
# fee_type

# smf.ols('ytm_gross_profit_amt ~ is_operation', data = sap).fit().summary() #ytm_gross_profit_amt

# smf.ols('ytm_gross_profit_amt ~ has_vas  ', data = sap).fit().summary() 

In [443]:
# sap.columns

In [447]:

# 不同象限WMS客户，其inbound/outbound量是否有显著区别？
# smf.ols(
#     'ytm_gross_profit_amt ~ inbound_line_count +\
#          inbound_receive_qty + outbound_line_count + outbound_shipped_qty ' # + stock_turn_monthly + location_usage_rate
#          , data = sap).fit().summary()

# smf.ols('ytm_gross_profit_amt ~ has_vas  ', data = sap).fit().summary()